In [251]:
import pandas as pd
import pyodbc


In [260]:
#function to split data set
def dataSplitter(path_csv,perJSon,perDataBase):

    data = pd.read_csv(path_csv)

    #répartition aléatoire des données. 
    data_randed = data.sample(frac=1)

    #len of data 
    lenOfData = len(data_randed)

    if(perJSon + perDataBase) == 100:

        p1 = round((lenOfData * perJSon) / 100)
        p2 = lenOfData - p1

        return data_randed[:p1], data_randed[p1:],[]
        #display(lenOfData,f'{perJSon}% = {p1}',f'{perDataBase}% = {p2}', p1+p2)
        
    elif(perJSon + perDataBase) < 100:
        p1 = round((lenOfData * perJSon) / 100)
        p2 = round((lenOfData * perDataBase) / 100)
        #p3 = lenOfData - (p1 + p2)

        return data_randed[:p1],data_randed[p1:p1+p2],data_randed[p1+p2:]
        #display(lenOfData,f'{perJSon}% = {p1}',f'{perDataBase}% = {p2}', f'{100 - (perDataBase + perJSon)}% = {p3}',p1+p2+p3)
    else:
        raise Exception('Saisir un pourcentage valide!')


def goDataToJSon(dataToJson,fileName):
    dataToJson.to_json(path_or_buf=f'data/{fileName}.json',orient='records')

def goDataToCsv(dataToCsv,fileName):
    dataToCsv.to_csv(f'data/{fileName}.csv')

def goDataToDataBase(dataToDb):
    #coonection to db
    cnxn = pyodbc.connect('Driver={SQL Server};'
                      'Server=XXXXXX-XXX\SQLEXPRESS;'
                      'Database=splitterdb;'
                      'Trusted_Connection=yes;')

    cursor = cnxn.cursor()
    for index,row in dataToDb.iterrows():
        cursor.execute('INSERT INTO contributors VALUES(?,?,?)',row['full_name'],row['name'],row['commits'])
    cnxn.commit()
    cnxn.close()
    

try:
    allDataFrame = dataSplitter('repo_contributors_05.csv',perJSon=30,perDataBase=20)
    
    dataToJson,dataToDatabase,dataToCsv = allDataFrame

    '''
        Création du fichier JSON
    '''
    if(len(dataToJson) > 0):
        goDataToJSon(dataToJson,'data')

    '''
        Création du fichier CSv
    '''
    if((len(dataToCsv)) > 0):
        goDataToCsv(dataToCsv,'data')

    '''
        Transfert vers la base de données
    '''
    if(len(dataToDatabase) > 0):
        goDataToDataBase(dataToDatabase)

except Exception as e:
    print(e.args[0])
